# Query the database

In [6]:
import pymongo
import pandas as pd
import sshtunnel
import numpy as np
from datetime import datetime, timedelta
from collections import defaultdict

In [7]:
server_ip = open("server.ip").read().strip()
database = "Windows-DESKTOP-3HVBJEA"  # SM2
sample_interval = '1H' # '1H' or '1D' 
# list_products = ["메리퀸", "죽향실온", "죽향저장" ]

In [8]:
def custom_avg(row):
    valid_values = [value for value in row if not pd.isnull(value) and value != 0]
    return np.mean(valid_values) if valid_values else None

In [9]:
with sshtunnel.open_tunnel((server_ip, 2222),
                           ssh_username='andrew',
                           ssh_password='a',
                           remote_bind_address=('127.0.0.1', 27017),
                           local_bind_address=('127.0.0.1', 37017)
                           ) as tunnel:
    # Construct MongoDB URI with the local bind address port
    mongo_uri = f"mongodb://localhost:{37017}"

    # connect to mongo uri
    client = pymongo.MongoClient(mongo_uri)

    db = client[database]

    # Collections
    com_collection = db["COM"]
    com4_collection = db["COM4"]
    com5_collection = db["COM5"]
    harvest_data_collection = db["HARVEST_DATA"]
    
    # Get all name of products in harvest_data_collection
    list_products = harvest_data_collection.distinct("Product Name")

    # Cache for sensor data to avoid repeated queries (keys are date and product name)
    sensor_data_cache = {} # {(date_start, date_end, product_name): aggregated_data}
    aggregation="AVG"
    
    for product_name in list_products:
        date_start = datetime.strptime("2024-04-04", "%Y-%m-%d")
        com_type = com_collection.find_one({product_name: {"$exists": True}}, {product_name: 1})
        if not com_type:
            continue
        result = []

        harvest_data = harvest_data_collection.find({"Product Name": product_name}).sort("Date", pymongo.ASCENDING)
        # Group by Date and Product Name
        grouped_data = defaultdict(list)
        for harvest in harvest_data:
            date = harvest["Date"]
            name = harvest["Product Name"]
            # rename Date to Datetime
            harvest["Datetime"] = harvest.pop("Date")
            # harvest["Datetime"] = datetime.strptime(harvest["Datetime"], "%Y-%m-%d")
            grouped_data[(date, name)].append(harvest)
        harvest_days = [group[0] for group in grouped_data.keys()]
        harvest_days.sort()
        
        # Building the final data
        for date in harvest_days:
            date_end = datetime.strptime(date, "%Y-%m-%d") + timedelta(days=1)
            harvest_data = pd.DataFrame(grouped_data[(date, product_name)]) # list of harvest data for the same date
            # # check if sensor data is already in cache
            # if (date_start, date_end, product_name) in sensor_data_cache:
            #     result.append({**harvest, **sensor_data_cache[(date_start, date_end, product_name)]})
            #     continue
            com_str = com_type[product_name]
            # Get sensor data from the corresponding COM collection
            sensor_data = db[com_str].find({
                "Datetime": {"$gte": date_start, "$lt": date_end}
            }).sort("Datetime", pymongo.ASCENDING)
            sensor_df = pd.DataFrame(sensor_data)
            if sensor_df.empty:
                print(f"Empty sensor data for {product_name} on {date}")
                # keep appending to result without sensor data (only harvest data)
                union_entry = harvest_data
            else:
                sensor_df.set_index('Datetime', inplace=True)
                sensor_df = sensor_df.drop(["_id"], axis=1)
                harvest_data.drop(["_id"], axis=1, inplace=True)
                resampled = sensor_df.resample(sample_interval)
                resampled_avg = resampled[sensor_df.columns].apply(lambda group: group.apply(custom_avg))
                resampled_avg.reset_index(inplace=True)
                resampled_avg["Datetime"] = pd.to_datetime(resampled_avg["Datetime"])
                harvest_data["Datetime"] = pd.to_datetime(harvest_data["Datetime"])
                # Merge harvest data with sensor data
                # union_entry = pd.merge_asof(resampled_avg, harvest_data, on="Datetime", direction="nearest")
                union_entry = pd.merge(harvest_data, resampled_avg, on='Datetime', how='outer')
            date_start = date_end
                
            result.append(union_entry)

        print(f"Product: {product_name}")
        # export to csv
        df = pd.concat(result)
        # sort by Date
        df["Datetime"] = pd.to_datetime(df["Datetime"])
        df = df.sort_values(by=["Datetime"])
        # remove the duplicate rows
        # df = df.drop_duplicates(subset=["Datetime"], keep='first')
        df.to_csv(f'{product_name}_{sample_interval}.csv', index=False, encoding='utf-8-sig')

    final_data = result
    client.close()

Product: 메리퀸
Product: 죽향실온
Product: 죽향저장


In [12]:
df["Datetime"] = pd.to_datetime(df["Datetime"])
df = df.sort_values(by=["Datetime"])

In [13]:
df

,_id,Product Name,Section,Plant,Fruit vertical length,Fruit horizontal length,Fruit weight,Hardness,Sweetness,Acidity,...,Remarks,Datetime,EC,Humidity,Sun,pH,Solid_Temperature,Temperature,CO2,Solid_Moisture
0,676e499d87af904bab15fa7b,메리퀸,2.0,3.0,38.53,28.75,10.98,NaN,16.1,0.75,...,과숙,2024-06-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,676e499d87af904bab15fa7c,메리퀸,3.0,1.0,42.76,32.61,15.72,210.0,11.8,0.63,...,NaN,2024-06-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,676e499d87af904bab15fa7d,메리퀸,4.0,1.0,NaN,NaN,7.29,NaN,NaN,NaN,...,NaN,2024-06-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,676e499d87af904bab15fa7e,메리퀸,4.0,2.0,NaN,NaN,5.22,NaN,NaN,NaN,...,NaN,2024-06-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,676e499d87af904bab15fa7f,메리퀸,4.0,3.0,NaN,NaN,3.63,NaN,NaN,NaN,...,NaN,2024-06-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,NaN,메리퀸,1.0,3.0,39.07,29.61,12.60,139.7,13.7,0.50,...,6.0,2024-12-06,NaN,78.251525,15604.631423,15632.179688,NaN,19.54703,777.450147,810.820312
1,NaN,메리퀸,1.0,4.0,33.54,26.03,10.11,143.8,11.2,0.54,...,6.0,2024-12-06,NaN,78.251525,15604.631423,15632.179688,NaN,19.54703,777.450147,810.820312
2,NaN,메리퀸,1.0,5.0,NaN,NaN,7.35,NaN,NaN,NaN,...,6.0,2024-12-06,NaN,78.251525,15604.631423,15632.179688,NaN,19.54703,777.450147,810.820312
3,NaN,메리퀸,3.0,2.0,42.18,32.35,16.07,152.5,13.2,0.53,...,6.0,2024-12-06,NaN,78.251525,15604.631423,15632.179688,NaN,19.54703,777.450147,810.820312


In [7]:
with sshtunnel.open_tunnel((server_ip, 2222),
                           ssh_username='andrew',
                           ssh_password='a',
                           remote_bind_address=('127.0.0.1', 27017),
                           local_bind_address=('127.0.0.1', 37017)
                           ) as tunnel:
    # Construct MongoDB URI with the local bind address port
    mongo_uri = f"mongodb://localhost:{37017}"

    # connect to mongo uri
    client = pymongo.MongoClient(mongo_uri)

    db = client[database]

    # Collections
    com_collection = db["COM"]
    com4_collection = db["COM4"]
    com5_collection = db["COM5"]
    harvest_data_collection = db["HARVEST_DATA"]

    # Cache for sensor data to avoid repeated queries (keys are date and product name)
    sensor_data_cache = {}
    aggregation="AVG"

    result = []

    def custom_avg(row):
        valid_values = [value for value in row if not pd.isnull(value) and value != 0]
        return np.mean(valid_values) if valid_values else None

    # Iterate over HARVEST_DATA
    for harvest in harvest_data_collection.find().sort("Date", pymongo.ASCENDING):
        product_name = harvest["Product Name"]
        date = datetime.strptime(harvest["Date"], "%Y-%m-%d")

        # Determine COM collection (COM4 or COM5)
        com_type = com_collection.find_one({product_name: {"$exists": True}}, {product_name: 1})
        if not com_type:
            continue
        com_str = com_type[product_name]
        # print(f"Checking date: {date}")
        # print(f"Product: {product_name}, COM Collection: {com_collection_name}")
        # check if sensor data is already in cache
        if (date, product_name) in sensor_data_cache:
            result.append({**harvest, **sensor_data_cache[(date, product_name)]})
            continue
        # Get sensor data from the corresponding COM collection
        sensor_data = db[com_str].find({
            # "Datetime": {"$gte": starting_date, "$lt": date}
            "Datetime": {"$gte": date, "$lt": date + timedelta(days=1)}
        })

        # # export sensor_data to csv
        # print(f"Number of sensor data: {len(list(sensor_data))}")
        sensor_df = pd.DataFrame(sensor_data)
        # sensor_df.to_csv(f'{product_name}.csv', index=False, encoding='utf-8-sig')
        # print(sensor_df.columns)
        # print(sensor_df.head())
        if sensor_df.empty:
            print(f"Empty sensor data for {product_name} on {date}")
            # keep appending to result without sensor data (only harvest data)
            union_entry = harvest
        else:
            avg = sensor_df.drop(["_id", "Datetime"], axis=1).apply(custom_avg).fillna(0)
            # print(avg)

            # Combine with HARVEST_DATA
            aggregated_data = avg.to_dict()
            sensor_data_cache[(date, product_name)] = aggregated_data
            union_entry = {**harvest, **aggregated_data}
            starting_date = date
        result.append(union_entry)

        # break

    final_data = result
    client.close()

Empty sensor data for 메리퀸 on 2024-11-29 00:00:00
Empty sensor data for 죽향저장 on 2024-11-29 00:00:00


In [8]:
# Convert to DataFrame 
df = pd.DataFrame(final_data)
df

,_id,Date,Product Name,Section,Plant,Fruit vertical length,Fruit horizontal length,Fruit weight,Hardness,Sweetness,...,Color b,Remarks,EC,Humidity,Sun,pH,Solid_Temperature,Temperature,CO2,Solid_Moisture
0,676e499d87af904bab15fa7b,2024-06-24,메리퀸,2,3,38.53,28.75,10.98,NaN,16.1,...,27.21,과숙,0.000000,75.509244,15577.452953,0.000000,0.0,20.254857,860.370872,889.453125
1,676e499d87af904bab15fa7c,2024-06-24,메리퀸,3,1,42.76,32.61,15.72,210.0,11.8,...,29.74,NaN,0.000000,75.509244,15577.452953,0.000000,0.0,20.254857,860.370872,889.453125
2,676e499d87af904bab15fa7d,2024-06-24,메리퀸,4,1,NaN,NaN,7.29,NaN,NaN,...,28.51,NaN,0.000000,75.509244,15577.452953,0.000000,0.0,20.254857,860.370872,889.453125
3,676e499d87af904bab15fa7e,2024-06-24,메리퀸,4,2,NaN,NaN,5.22,NaN,NaN,...,29.35,NaN,0.000000,75.509244,15577.452953,0.000000,0.0,20.254857,860.370872,889.453125
4,676e499d87af904bab15fa7f,2024-06-24,메리퀸,4,3,NaN,NaN,3.63,NaN,NaN,...,NaN,NaN,0.000000,75.509244,15577.452953,0.000000,0.0,20.254857,860.370872,889.453125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,676e49bf87af904bab15fc46,2024-12-06,죽향저장,3,3,50.34,34.96,23.89,150.5,12.7,...,27.50,5,1.269949,81.329227,2986.958743,0.932005,0.0,19.015062,728.586070,55.799999
426,676e49bf87af904bab15fc47,2024-12-06,죽향저장,3,4,43.40,33.44,20.02,179.2,11.6,...,28.57,5,1.269949,81.329227,2986.958743,0.932005,0.0,19.015062,728.586070,55.799999
427,676e49bf87af904bab15fc48,2024-12-06,죽향저장,4,1,NaN,NaN,4.86,NaN,NaN,...,NaN,5,1.269949,81.329227,2986.958743,0.932005,0.0,19.015062,728.586070,55.799999
428,676e49bf87af904bab15fc49,2024-12-06,죽향저장,4,2,40.02,26.02,11.24,161.5,10.4,...,25.87,5,1.269949,81.329227,2986.958743,0.932005,0.0,19.015062,728.586070,55.799999


In [9]:
# sort by Date, Product Name, Section, Plant
df.drop(["_id"], axis=1, inplace=True)
df = df.sort_values(by=["Date", "Product Name", "Section", "Plant"])
df

,Date,Product Name,Section,Plant,Fruit vertical length,Fruit horizontal length,Fruit weight,Hardness,Sweetness,Acidity,...,Color b,Remarks,EC,Humidity,Sun,pH,Solid_Temperature,Temperature,CO2,Solid_Moisture
0,2024-06-24,메리퀸,2,3,38.53,28.75,10.98,NaN,16.1,0.75,...,27.21,과숙,0.000000,75.509244,15577.452953,0.000000,0.0,20.254857,860.370872,889.453125
1,2024-06-24,메리퀸,3,1,42.76,32.61,15.72,210.0,11.8,0.63,...,29.74,NaN,0.000000,75.509244,15577.452953,0.000000,0.0,20.254857,860.370872,889.453125
2,2024-06-24,메리퀸,4,1,NaN,NaN,7.29,NaN,NaN,NaN,...,28.51,NaN,0.000000,75.509244,15577.452953,0.000000,0.0,20.254857,860.370872,889.453125
3,2024-06-24,메리퀸,4,2,NaN,NaN,5.22,NaN,NaN,NaN,...,29.35,NaN,0.000000,75.509244,15577.452953,0.000000,0.0,20.254857,860.370872,889.453125
4,2024-06-24,메리퀸,4,3,NaN,NaN,3.63,NaN,NaN,NaN,...,NaN,NaN,0.000000,75.509244,15577.452953,0.000000,0.0,20.254857,860.370872,889.453125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,2024-12-06,죽향저장,3,3,50.34,34.96,23.89,150.5,12.7,0.62,...,27.50,5,1.269949,81.329227,2986.958743,0.932005,0.0,19.015062,728.586070,55.799999
426,2024-12-06,죽향저장,3,4,43.40,33.44,20.02,179.2,11.6,0.58,...,28.57,5,1.269949,81.329227,2986.958743,0.932005,0.0,19.015062,728.586070,55.799999
427,2024-12-06,죽향저장,4,1,NaN,NaN,4.86,NaN,NaN,NaN,...,NaN,5,1.269949,81.329227,2986.958743,0.932005,0.0,19.015062,728.586070,55.799999
428,2024-12-06,죽향저장,4,2,40.02,26.02,11.24,161.5,10.4,0.62,...,25.87,5,1.269949,81.329227,2986.958743,0.932005,0.0,19.015062,728.586070,55.799999


In [11]:
df.to_csv('final_data.csv', index=False, encoding='utf-8-sig')